In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, MatchValue

/home/cheetos/Bureau/Rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
def load_embedding_model(model_name="intfloat/multilingual-e5-large-instruct"):
    """
    Charge le modèle d'embeddings et le tokenizer depuis Hugging Face.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    return tokenizer, model, device






def vectorize_query(query: str, tokenizer, model, device):
    """
    Vectorise une requête utilisateur pour le retriever.
    """
    query = f"query: {query}"

    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state.mean(dim=1)

    return embedding.cpu().numpy()






def search_qdrant(query_vector, client, collection_name, top_k=5):
    """
    Effectue une recherche dans la collection Qdrant à partir d'un vecteur de requête.
    """
    query_vector = query_vector[0].tolist()

    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k,
        with_payload=True
    )

    for result in results:
        payload = result.payload
        chunk_text = payload.get("texte", "Texte non disponible")
        print("Chunk trouvé :", chunk_text)
        print("Sujet :", payload.get("sujet", "Non spécifié"))
        print("Score :", result.score)
        print("-" * 50)

    return results







def main_retrieve(query: str, collection_name="GameRag"):
    """
    Charge le modèle, vectorise la requête et effectue la recherche dans Qdrant.
    """
    tokenizer, model, device = load_embedding_model()

    client = QdrantClient(url="http://localhost:6333")

    query_vector = vectorize_query(query, tokenizer, model, device)

    results = search_qdrant(query_vector, client, collection_name)
    
    return results

In [28]:
a = main_retrieve("Quelles statistiques sont suivies pendant mes sessions de jeu ?")

Chunk trouvé : Réponses aux questions sur les statistiques détaillées :
Q: "Peux-tu me donner une analyse détaillée de mes dernières sessions ?"
R: "Analyse des 10 dernières sessions : score moyen : 850 points, meilleur score : 1200 points (session n°7), niveau maximum atteint : 7, opérations les plus réussies : additions (92%), points à améliorer : divisions (76%), temps moyen par problème : 6.5 secondes, Pprogression globale : +15% par rapport à la semaine précédente."


Sujet : Réponses aux questions sur les statistiques détaillées
Score : 0.86762697
--------------------------------------------------
Chunk trouvé : Système de score :
Le joueur gagne des points en répondant correctement aux questions. Le temps de réponse est pris en compte, une jauge de progression est présente. Les statistiques suivantes sont suivies : Nombre de réponses correctes, nombre total de tentatives, temps de réponse moyen, précision par type d'opération.


Sujet : Système de score
Score : 0.8511298
-------

/tmp/ipykernel_36759/1427904616.py:41: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
